# Project EDA and Data Pre-processing

## Set-up

In [1]:
import os
from os.path import join  
import requests
import zipfile
import tarfile
import shutil
import math
import json
import time
import sys
import cv2
import string
import re
import subprocess
import hashlib
import numpy as np
import pandas as pd
from glob import glob
import collections
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.utils.layer_utils import count_params

# sklearn
from sklearn.model_selection import train_test_split

# Tensorflow Hub
import tensorflow_hub as hub

# Colab auth
from google.colab import auth
from google.cloud import storage

### GitHub Integration

In [3]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT, force_remount=True)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [16]:
# Integrate GitHub

# each group member should add their own ENV_PATH here
# comment out other people's ENV_PATHs
# Steve:
ENV_PATH = "/content/drive/MyDrive/adv_practical_data_science/local-repo/AC215_KKST/.env"
# Matt:
# Shih-ye:
# Al:
# Ed:

# load environment variables
with open(ENV_PATH) as env:
  env_text = env.read()
env_list = env_text.split("\n")
PROJECT_PATH = env_list[0]
GIT_PATH = env_list[1]

# expand paths
REPO_PATH = PROJECT_PATH + '/AC215_KKST'
NOTEBOOK_DIR_PATH = REPO_PATH + '/notebooks'

# change directory to the local repo's notebook folder
%cd '{NOTEBOOK_DIR_PATH}'

/content/drive/MyDrive/adv_practical_data_science/local-repo/AC215_KKST/notebooks


#### Cells for updating local/remote repos

In [17]:
# check statuses of the files you changed
# this will give a list of files currently in the "head"
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ../Makefile
	modified:   ../README.md
	modified:   ../models/.gitkeep
	modified:   .gitkeep
	modified:   ../references/.gitkeep
	modified:   ../requirements.txt
	modified:   ../setup.py
	modified:   ../src/__init__.py
	modified:   ../src/build_features.py
	modified:   ../test_project.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	EDA.ipynb
	template-colab-git-integration.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
